# We will use colaaborative filtering to do it

### Used mean imputation for filling out null values
### Used refilling of output to input and again predicting the movie rating
### Since data is very very sparse its hard to make a predictor 
### This code is using simple prediction and also refilling prediction

In [734]:
import pandas as pd
import numpy as np
import random
random.seed(10)

In [735]:
# Data Read

data = pd.read_csv("Amazon - Movies and TV Ratings.csv", sep =",")

In [736]:
data.shape

(4848, 207)

In [737]:
data.head()

,user_id,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,A3R5OBKS7OM2IR,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AH3QC2PC1VTGP,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3LKP6WPMP9UKX,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVIY68KEPQ5ZD,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1CV1WROP5KTTW,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [738]:
data.isnull().sum()   ### Number of null values in each column

user_id        0
Movie1      4847
Movie2      4847
Movie3      4847
Movie4      4846
            ... 
Movie202    4842
Movie203    4847
Movie204    4840
Movie205    4813
Movie206    4835
Length: 207, dtype: int64

In [739]:
# Number of ratings in each column

rating_count_per_movie = data.drop("user_id", axis =1).count(axis=0)    
rating_count_per_movie

Movie1       1
Movie2       1
Movie3       1
Movie4       2
Movie5      29
            ..
Movie202     6
Movie203     1
Movie204     8
Movie205    35
Movie206    13
Length: 206, dtype: int64

### Movie with most rating

In [740]:
rating_count_per_movie.idxmax(), rating_count_per_movie.max(), data.Movie127.count()

('Movie127', 2313, 2313)

#### Movie "Movie127" have most rating that is 2313

### Other way to do it

In [741]:
rating_count_per_movie_df = pd.DataFrame(rating_count_per_movie)

In [742]:
rating_count_per_movie_df.head()

,0
Movie1,1
Movie2,1
Movie3,1
Movie4,2
Movie5,29


In [743]:
rating_count_per_movie_df.iloc[:,0].sort_values(axis =0, ascending = False)

Movie127    2313
Movie140     578
Movie16      320
Movie103     272
Movie29      243
            ... 
Movie68        1
Movie69        1
Movie145       1
Movie71        1
Movie1         1
Name: 0, Length: 206, dtype: int64

#### Movie "Movie127" have most rating that is 2313

In [744]:
data_info = data.describe()

### Top 5 most rated Movies

In [745]:
data.mean().sort_values(axis =0, ascending = False).head()

Movie1      5.0
Movie55     5.0
Movie131    5.0
Movie132    5.0
Movie133    5.0
dtype: float64

###  Movies with least audience
#### We will take total count of user ratings per movie
#### And which movie is rated by less people will be output

In [746]:
data.drop("user_id", axis =1).count(axis=0).sort_values(axis =0, ascending = True).head()    

Movie1      1
Movie71     1
Movie145    1
Movie69     1
Movie68     1
dtype: int64

In [747]:
#data = data.fillna(0)
#data.shape

In [748]:
data['user_id'].unique().shape[0]

4848

In [749]:
data.isnull().sum().any()

True

In [750]:
from sklearn.preprocessing import LabelEncoder

In [751]:
from sklearn.preprocessing import LabelEncoder
lenc = LabelEncoder()
lenc.fit(data['user_id'])
data['user_id'] = lenc.transform(data['user_id'])

In [752]:
(data.dtypes == object).any()

False

### Prepare data in the format for recommender system

In [753]:
data.head()

,user_id,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,3549,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4200,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3339,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4689,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,458,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##  Code for data prepration

In [754]:
# Mapping every id with every movie and their rating
# So every id will have 206 movie entries and 206 ratings

lst_with_all_possible_combination_of_user_and_movie_ratings = []
all_movie_ids = data.drop(["user_id"], axis =1).columns

for indd,roww in data.iterrows():
    # i is index and j is df of single row
    #print(j.Movie1)
    temp1 = roww.user_id
    
    for k in all_movie_ids:
        #print(k)
        lst_with_all_possible_combination_of_user_and_movie_ratings.append([temp1,k, roww[k]])
    #print(temp2)
    
pd.DataFrame(lst_with_all_possible_combination_of_user_and_movie_ratings)    
    

,0,1,2
0,3549.0,Movie1,5.0
1,3549.0,Movie2,5.0
2,3549.0,Movie3,NaN
3,3549.0,Movie4,NaN
4,3549.0,Movie5,NaN
...,...,...,...
998683,4242.0,Movie202,NaN
998684,4242.0,Movie203,NaN
998685,4242.0,Movie204,NaN
998686,4242.0,Movie205,NaN


In [755]:
Df = pd.DataFrame(lst_with_all_possible_combination_of_user_and_movie_ratings)
Df.columns = ["user_id", "movie_id", "ratings"]
Df.shape

(998688, 3)

In [756]:
trans_Df = Df.dropna()

In [757]:
trans_Df.shape

(5000, 3)

#### Label encode movie id

In [758]:
le = LabelEncoder()
le.fit(trans_Df['movie_id'])
trans_Df['movie_id'] = le.transform(trans_Df['movie_id'])

C:\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [759]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(trans_Df, test_size = .3)

In [760]:
train.shape, test.shape

((3500, 3), (1500, 3))

In [761]:
train.head()

,user_id,movie_id,ratings
837735,222.0,46,5.0
978280,2177.0,104,3.0
829495,2011.0,46,5.0
704028,3960.0,31,2.0
597320,851.0,31,5.0


### Initialize training and test container matrix

In [762]:
training_matrix = np.zeros(data.shape)
test_matrix = np.zeros(data.shape)
training_matrix.shape

(4848, 207)

In [763]:
for line in train.itertuples():
    print(line)
    break

Pandas(Index=837735, user_id=222.0, movie_id=46, ratings=5.0)


#### Filling train and test matrices with available ratings

In [764]:
for line in train.itertuples():
    training_matrix[int(line[1]), line[2]] = line[3]
training_matrix  

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [765]:
for line in test.itertuples():
    test_matrix[int(line[1]), line[2]] = line[3]
test_matrix[test_matrix>4]

array([5., 5., 5., ..., 5., 5., 5.])

In [766]:
i, j = np.where(trans_Df>4)
i, j 

(array([   0,    0,    1, ..., 4999, 4999, 4999], dtype=int64),
 array([0, 2, 0, ..., 0, 1, 2], dtype=int64))

In [767]:
trans_Df.iloc[0,0]

3549.0

#### Calculating similarity and prediction

In [768]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(training_matrix, metric='cosine')
item_similarity = pairwise_distances(training_matrix.T, metric='cosine')

In [769]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #print("mean_user_rating", ratings.shape, mean_user_rating[:, np.newaxis].shape)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        #print("ratings_diff", ratings_diff.shape)
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [770]:
#item_prediction = predict(test_matrix, item_similarity, type='item')
user_prediction = predict(test_matrix, user_similarity, type='user')

#### Calculating RMSE value

In [771]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    #print(prediction)
    prediction = prediction[ground_truth.nonzero()].flatten() 
    #print(prediction.shape)
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    #print(prediction.shape)
    return sqrt(mean_squared_error(prediction, ground_truth))

In [772]:
#print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_matrix)))
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_matrix)))

User-based CF RMSE: 4.226894192214015


In [773]:
user_prediction[6,187]

0.01767316603028518

### Since some values are negative prediction we will scale it in between range(0-5)

In [774]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (0,5))
user_prediction_scaled = scaler.fit_transform(user_prediction) 


In [775]:
user_prediction_scaled[user_prediction_scaled>4].shape

(1814,)

In [776]:
#print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_matrix)))
print('User-based CF RMSE: ' + str(rmse(user_prediction_scaled, test_matrix)))

User-based CF RMSE: 3.719185599481293


In [777]:
user_prediction.shape

(4848, 207)

## RMSE is 3.59 which is high, we have to reduce it

# Model 2

## Here i am gonna use refilling method to create new input using predicted values and will use that input to predict new output

### creating new data using refilling using User_predictions

In [778]:
data_new = np.zeros(data.shape)
data_new.shape

(4848, 207)

In [779]:
def nan_check(val):
    return val !=val

In [780]:
np.nan == np.nan   ## NaN value check

False

In [781]:
le = LabelEncoder()
le.fit(Df['movie_id'])
Df['movie_id'] = le.transform(Df['movie_id'])
Df.shape

(998688, 3)

### Here we will use original data to fill the available values and remaining will be filled using User_predictions

In [782]:
for line in Df.itertuples():
    if(nan_check(line[3])):
        data_new[int(line[1]), line[2]] = user_prediction[int(line[1]), line[2]]
    else:
        data_new[int(line[1]), line[2]] = line[3]
        
data_new 

array([[-0.00596539, -0.0070935 , -0.00619101, ..., -0.0070935 ,
        -0.0070935 ,  0.        ],
       [-0.00813481, -0.00967939, -0.00844372, ..., -0.00967939,
        -0.00967939,  0.        ],
       [-0.00813481, -0.00967939, -0.00844372, ..., -0.00967939,
        -0.00967939,  0.        ],
       ...,
       [-0.00548654, -0.00652409, -0.00569405, ..., -0.00652409,
        -0.00652409,  0.        ],
       [-0.00813481, -0.00967939, -0.00844372, ..., -0.00967939,
        -0.00967939,  0.        ],
       [-0.00813481, -0.00967939, -0.00844372, ..., -0.00967939,
        -0.00967939,  0.        ]])

In [783]:
data_new.shape
(data_new>4).any()

True

In [784]:
(data_new==5).sum()

3659

In [785]:
np.where(Df.ratings==5)

(array([     0,      1,    415, ..., 998275, 998481, 998687], dtype=int64),)

In [786]:
Df.iloc[0,:]

user_id     3549.0
movie_id       0.0
ratings        5.0
Name: 0, dtype: float64

In [787]:
data_new[3549,0]   ### Values are inserted at right places

5.0

### Split in to train and test

In [788]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data_new, test_size = .3)

#### Make shape to input shape

In [789]:
training_matrix = np.zeros(data.shape)
test_matrix = np.zeros(data.shape)
training_matrix.shape

(4848, 207)

### Fill up the values using train and test

In [790]:
training_matrix[0: train.shape[0], :] = train


In [791]:
(train>4).sum()

2550

In [792]:
test_matrix[0: test.shape[0], :] = test

### Calculating similarity and predictions

In [793]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(training_matrix, metric='cosine')
#item_similarity = pairwise_distances(training_matrix.T, metric='cosine')

In [794]:
#item_prediction = predict(test_matrix, item_similarity, type='item')
user_prediction = predict(test_matrix, user_similarity, type='user')

In [795]:
(user_prediction>0).any()

True

In [796]:
#print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_matrix)))
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_matrix)))

User-based CF RMSE: 0.30439449098660276


### RMSE is .3 but some values are less than zero, So we will rescale

In [797]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (0,5))
user_prediction_scaled = scaler.fit_transform(user_prediction) 


In [798]:
#print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_matrix)))
print('User-based CF RMSE: ' + str(rmse(user_prediction_scaled, test_matrix)))

User-based CF RMSE: 0.7874370943428727


## RMSE is .78 which is improved from 3.7

In [800]:
## We will use other methods for improving results
## Like a predictor with unsupervised learning